In [10]:
from grafos import Accion
from grafos import Estado
from grafos import Nodo
from grafos import Problema

Crear métodos auxiliares

In [11]:
# Identifica el estado inicial (el centro de distribución)
def crear_nodo_raiz(problema):
    estado_raiz = problema.estado_inicial
    acciones_raiz = {}
    if estado_raiz.nombre in problema.acciones.keys():
        acciones_raiz = problema.acciones[estado_raiz.nombre]
    raiz = Nodo(estado_raiz, None, acciones_raiz, None)
    return raiz

# A partir del nodo raiz identifaca de donde viene y hacia donde puede ir
def crear_nodo_hijo(problema, padre, accion):
    nuevo_estado = problema.resultado(padre.estado, accion)
    acciones_nuevo = {}
    if nuevo_estado.nombre in problema.acciones.keys():
        acciones_nuevo = problema.acciones[nuevo_estado.nombre]
    hijo = Nodo(nuevo_estado, accion, acciones_nuevo, padre)
    padre.hijos.append(hijo)
    return hijo

# Nos muestra la solución
def muestra_solucion(objetivo = None):
    if not objetivo:
        print('No hay solución')
        return
    nodo = objetivo
    while nodo:
        msg = 'Estado: {0}'
        print(msg.format(nodo.estado.nombre))
        if nodo.accion:
            msg = "<----{0}---->"
            print(msg.format(nodo.accion.nombre))
        nodo = nodo.padre

Crear la función que ejecuta el problema

In [12]:
# Llama a los métodos auxiliares, aquí le entregamos los datos 
# al algoritmo de busqueda
def amplitud(problema):
    raiz = crear_nodo_raiz(problema)
    if problema.es_objetivo(raiz.estado):
        return raiz
    frontera = [raiz,]
    explorados = set()
    while True:
        print('frontera: ', [nodo.estado.nombre for nodo in frontera])
        print('explorados: ', [estado.nombre for estado in explorados])
        if not frontera:
            return None
        nodo = frontera.pop(0)
        print('escoge: ', nodo.estado.nombre)
        print('------------')
        explorados.add(nodo.estado)
        if not nodo.acciones:
            continue
        for nombre_accion in nodo.acciones.keys():
            accion = Accion(nombre_accion)
            hijo = crear_nodo_hijo(problema, nodo, accion)
            estados_frontera = [nodo.estado for nodo in frontera]
            if (hijo.estado not in explorados and hijo.estado not in estados_frontera):
                if problema.es_objetivo(hijo.estado):
                    return hijo
                frontera.append(hijo)

# Crear la estructura del problema

In [17]:
if __name__ == '__main__':
    # vamos a crear las acciones que puede hacer el agente
    accN = Accion('N')
    accS = Accion('S')
    accE = Accion('E')
    accO = Accion('O')
    accNE = Accion('NE')
    accSE = Accion('SE')
    accSO = Accion('SO')
    accNO = Accion('NO')

    # Vamos a crear los estados
    cedi = Estado('cedi', [accSO, accSE])
    tienda_1 = Estado('tienda_1', [accS, accSE])
    tienda_2 = Estado('tienda_2', [accO, accS, accSE])
    tienda_3 = Estado('tienda_3', [accE])
    tienda_4 = Estado('tienda_4', [accE])
    tienda_5 = Estado('tienda_5', [accNE])
    parqueadero = Estado('parqueadero', [])

    # Acciones que puede hacer
    acciones = {'cedi':{'SE':tienda_2, 'SO':tienda_1},
                'tienda_1':{'S':tienda_3, 'SE':tienda_4},
                'tienda_2':{'O':tienda_1, 'S':tienda_4, 'SE':parqueadero},
                'tienda_3':{'E':tienda_4},
                'tienda_4':{'E':tienda_5},
                'tienda_5':{'NE':parqueadero}}

In [20]:
objetivo = [parqueadero]
problema = Problema(cedi, objetivo, acciones)
solucion = amplitud(problema)
muestra_solucion(solucion)

frontera:  ['cedi']
explorados:  []
escoge:  cedi
------------
frontera:  ['tienda_2', 'tienda_1']
explorados:  ['cedi']
escoge:  tienda_2
------------
Estado: parqueadero
<----SE---->
Estado: tienda_2
<----SE---->
Estado: cedi
